## Reading documents

In [1]:
import os

from langchain_community.document_loaders import PyPDFLoader

In [2]:
DATA_PATH = r'data\Robinson_Crusoe.pdf'

loader = PyPDFLoader(DATA_PATH)
document = loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
    add_start_index = True
)

chunks = text_splitter.split_documents(document)

len(document), len(chunks)

(390, 1769)

In [5]:
document = chunks[42]

print(document.page_content)
print(document.metadata)

These wise and sober thoughts continued all the while the storm lasted, and 
indeed some time after; but the next day the wind was abated, and the sea 
calmer, and I began to be a little inured to it; however, I was very grave for 
all that day, being also a little sea-sick still; but towards night the weather 
cleared up, the wind was quite over, and a charming fine evening followed; 
the sun went down perfectly clear, and rose so the next morning; and having
{'source': 'data\\Robinson_Crusoe.pdf', 'page': 9, 'start_index': 365}


## Embeddings

In [7]:
from langchain_chroma import Chroma

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [8]:
embedding_function = SentenceTransformerEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

c:\Users\denis\anaconda3\envs\qaenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\denis\anaconda3\envs\qaenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
db = Chroma.from_documents(chunks, embedding_function, persist_directory="./chroma_db")
# db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

In [32]:
query = "How Robinson Crusoe met the dog?"

docs = db.similarity_search_with_relevance_scores(query, k = 3)

c:\Users\denis\anaconda3\envs\qaenv\lib\site-packages\langchain_core\vectorstores\base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'page': 0, 'source': 'data\\Robinson_Crusoe.pdf', 'start_index': 0}, page_content='www.freeclassicebooks.com  \n1  \nRobinson Crusoe \n \nBy  \n \nDaniel Defoe \n \n \nwww.freeclassicebooks.com'), -18.17734341382535), (Document(metadata={'page': 1, 'source': 'data\\Robinson_Crusoe.pdf', 'start_index': 0}, page_content='www.freeclassicebooks.com  \n2 Contents \n \nChapter 1: Start In Life. ...................................................................................................... .....................3 \nChapter 2: Slavery And Escape .................................................................................................. ...........20 \nChapter 3: Wrecked On A Desert Island .......................................................................................... .....37'), -21.34064579460394),

In [33]:
docs

[(Document(metadata={'page': 0, 'source': 'data\\Robinson_Crusoe.pdf', 'start_index': 0}, page_content='www.freeclassicebooks.com  \n1  \nRobinson Crusoe \n \nBy  \n \nDaniel Defoe \n \n \nwww.freeclassicebooks.com'),
  -18.17734341382535),
 (Document(metadata={'page': 1, 'source': 'data\\Robinson_Crusoe.pdf', 'start_index': 0}, page_content='www.freeclassicebooks.com  \n2 Contents \n \nChapter 1: Start In Life. ...................................................................................................... .....................3 \nChapter 2: Slavery And Escape .................................................................................................. ...........20 \nChapter 3: Wrecked On A Desert Island .......................................................................................... .....37'),
  -21.34064579460394),
 (Document(metadata={'page': 306, 'source': 'data\\Robinson_Crusoe.pdf', 'start_index': 775}, page_content='him a little, I called him to me, and he